* The previous notebook began to run a little more slowly than desirable, so off we just into another one.
* Build a data modeling paradigm for practice.

In [26]:
from pandas import Series
import pandas as pd
import numpy as np
import scipy.stats
import math

def beta_1_hat(dat):
    x_m = x_mean(dat)
    y_m = y_mean(dat)
    beta_1_hat_num = sum([(x - x_m)*(y - y_m) for (x,y) in dat.iteritems()])
    beta_1_hat_denom = sum([(d[0] - x_m)**2 for d in dat.iteritems()])
    beta_1_hat = beta_1_hat_num / beta_1_hat_denom
    return beta_1_hat

def beta_0_hat(dat):
    beta_0_hat = y_mean(dat) - beta_1_hat(dat)*x_mean(dat)
    return beta_0_hat

def n(dat):
    return len(dat)

def x_vals(dat):
    return np.array(dat.index)

def x_mean(dat):
    return np.mean(x_vals(dat))

def y_mean(dat):
    return dat.mean()

def sse(dat):
    return sum([(d[1] - (beta_0_hat(dat) + beta_1_hat(dat)*d[0]))**2 for d in dat.iteritems()])

def sst(dat):
    return sum([(d[1] - y_mean(dat))**2 for d in dat.iteritems()])

def r_squared(dat):
    return 1 - sse(dat)/sst(dat)

def variance(dat):
    return sse(dat)/(n(dat) - 2)

def standard_deviation_estimator(dat):
    return variance(dat)**(1/2)

def beta_1_hat_standard_deviation_estimator(dat):
    return standard_deviation_estimator(dat)/math.sqrt(sum(x_vals(dat)**2) - (sum(x_vals(dat))**2)/n(dat))

def beta_1_hat_confidence_interval(dat, confidence=.95):
    t_rv = scipy.stats.t(n(dat)-2)
    t_value = t_rv.ppf(1 - (1 - confidence)/2)
    interval = (beta_1_hat(dat) - t_value * beta_1_hat_standard_deviation_estimator(dat),
                beta_1_hat(dat) + t_value * beta_1_hat_standard_deviation_estimator(dat))
    return interval

In [17]:
dat = Series({16.1:4.41,
              31.5:6.81,
              21.5:5.26,
              22.4:5.99,
              20.5:5.92,
              28.4:6.14, 
              30.3:6.84, 
              25.6:5.87, 
              32.7:7.03, 
              29.2:6.89, 
              34.7:7.87}
            )

In [27]:
y_mean(dat), x_mean(dat), beta_0_hat(dat), beta_1_hat(dat), r_squared(dat), beta_1_hat_standard_deviation_estimator(dat), beta_1_hat_confidence_interval(dat)

(6.275454545454545,
 26.627272727272725,
 2.2249345952600814,
 0.15211921970685938,
 0.88135230886566351,
 0.01860448731760117,
 (0.11003294546222395, 0.19420549395149481))

* A linear regression model in which we have a high degree of confidence can then be used to construct an estimate of the true mean of the population (as opposed to the mean of the population: a population by itself does not provide enough information to construct a confidence interval).
* $E(\hat{\beta}_0 + \hat{\beta}_1\cdot x^*) = \beta_0 + \beta_1 x^*$ (duh)
* $\sigma^2(\hat{\beta}_0 + \hat{\beta}_1\cdot x^*) = \sigma^2 \left(\frac{1}{n} + \frac{n(x^* - \bar{x})^2}{n\sum x_i^2 - (\sum x_i)^2}\right)$.
* Notice that variance increases as $x^*$ moves away from $\bar{x}$! This implies that we are more sure of our fit near the mean and less sure of it further away, which, of course, makes sense: small differences in slope cause large differences in value at extremes.
* $T = \frac{\hat{\beta}_0 + \hat{\beta}_1\cdot x^* - (\beta_0 + \beta_1\cdot x^*)}{s_{\hat{\beta}_0 + \hat{\beta}_1\cdot x^*}} \sim t[n-2](\cdot)$, and thus our confidence interval is $\hat{\beta}_0 + \hat{\beta}_1 x^* \pm t_{\alpha/2, n-2}\cdot s_{\hat{\beta}_0 + \hat{\beta}_1\cdot x^*}$.
* A property of the normal that follows from the stochasticly useful **Bonferroni inequality** is that given that we are $\alpha$ confident in one interval and $\beta$ confident in another one, we are $1 - (1 - \alpha + 1 - \beta)$ confident in both being simultaneously true. For instance if we are 99% confident that the value for y(2) is distributed within (12,24) and 99% confident that the value of y(4) is distributed within (20, 30) then we are 98% confident in the statement that both y(2) and y(4) are respectively in their intervals.

In [76]:
def estimated_x_variance(dat, estimated_x):
    # This method returns significant accumulated floating-point error.
    # Would be better to work with the logs of things, but I don't immediately know how to do that. Should learn that technique.
    return standard_deviation_estimator(dat)*math.sqrt(1/n(dat) + n(dat)*(estimated_x - x_mean(dat))**2/(n(dat)*sum([d[0]**2 for d in dat.iteritems()]) - sum([d[0] for d in dat.iteritems()])**2))

def estimated_x_interval(dat, estimated_x, confidence=.95):
    t_rv = scipy.stats.t(n(dat)-2)
    t_value = t_rv.ppf(1 - (1 - confidence)/2)
    interval = (beta_0_hat(dat) + beta_1_hat(dat)*estimated_x - t_value * estimated_x_variance(dat, estimated_x),
                beta_0_hat(dat) + beta_1_hat(dat)*estimated_x + t_value * estimated_x_variance(dat, estimated_x))
    return interval

In [70]:
dat = Series(
    {1000:220,
     1100:280,
     1200:350,
     1250:375,
     1300:450,
     1400:470,
     1450:500}
    )

In [77]:
estimated_x_interval(dat, 1200, confidence=.99)

(322.49947180995491, 378.34559861258015)

* Which solves example 12.14 on page 481!
* At this point I went back and derived the non-obvious parts of what I'd read from principles.
 * http://stats.stackexchange.com/questions/76444/why-is-chi-square-used-when-creating-a-confidence-interval-for-the-variance
* At this point I want to go back and rederive the linear modeling results so far, from first principles.
* ...done...

* The **standardized residual plot** is incredibly useful for checking the rightness of a regression.
* A distribution is **intrinsically linear** if it can, by some transformation, be made linear (and thus, regressable).
* Does that mean lognormal and other such transformable variables are intrinsically normal?
* Polynomial regression is usually only appropriate for up to a cubic.
* You can technically get 100% accuracy with n - 1 degrees of a polynomial regression.
* Again there is a formula for $r^2$, coefficient of multiple determination.
* A higher-degree regression will include lower-degree ones as special cases, and so will be strictly superior. So it will have a higher $r^2$, but the complexity, will mean that it might well be worse anyway!
* There is also an adjusted coefficient which weighs which is useful for weighing a higher-order regression against a lower one. If the adjusted $r^2$ compares poorly against the basic $r^2$ of the lower-polynomial model, then the lower model should be used, as not enough benefit is provided in the additional degree.
* How is the adjustment made, logically? Dunno-figure it out if I need it.
* Since with enough observations the regression error will be approximately normal, you can compute the boundaries against the polynomial approximation in the usual way. This nets you $\hat{\beta}_i \pm t_{\alpha/2,n-(k+1)}\cdot s_{\hat{\beta}_i}$ (where n is the number of observations and k the number of degrees).
* As with the linear model, there are also confidence intervals for the mean and for future predictions.

* **Multiple regression analysis** relates a single dependent variable $Y$ to several independent (predictor) values $x_k$. 
* So basically, multivariate regression.
* These models can be typified as having interaction (in which case you have $\beta_1 x_1 x_2$ type stuff) or no interaction (in which case all of the $x_1$ and $x_2$ terms are strictly seperate). The overall model looks something like $Y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon$
* Again there is a confidence interval construction and a hypothesis test.
* Skipping "Other issues"...